In [1]:
import os
import pandas as pd
from src.StreamPort.machine_learning.methods import MachineLearningEvaluateModelStabilityNative

rest_indices = 20
test_indices = 4

train_files = []
train_meta = []

path_to_test_records = "dev/error_lc_test_record.csv"

test_record = pd.read_csv(path_to_test_records) if os.path.exists(path_to_test_records) else None
test_record = test_record.sort_values("date") if test_record is not None else None

if test_record is not None and len(test_record) > rest_indices//test_indices:
    result_logs = []
    for date in test_record["date"]:
        filepath = f"dev/error_lc_test_{date}_classified_samples.csv"
        featurepath = f"dev/error_lc_test_{date}_features.csv"
        metapath = f"dev/error_lc_test_{date}_metadata.csv"
        result_logs.append(filepath) if os.path.exists(filepath) else print(f"No records for {date}")
        train_files.append(featurepath) if os.path.exists(featurepath) else print(f"No records for {date}")
        train_meta.append(metapath) if os.path.exists(metapath) else print(f"No records for {date}")
else:
    print("Not enough evidence of true inliers! Please run more tests for more data")


Not enough evidence of true inliers! Please run more tests for more data


c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

summary = pd.DataFrame()
train_data = pd.DataFrame()
metadata = pd.DataFrame()

for i in range(len(train_files)):
    feature_df = pd.read_csv(train_files[i])
    train_data = pd.concat([train_data, feature_df], ignore_index=True)

    log = pd.read_csv(result_logs[i])
    summary = pd.concat([summary, log], ignore_index=True)

    meta = pd.read_csv(train_meta[i])
    metadata = pd.concat([metadata, meta], ignore_index=True)


error_labeled_meta = metadata.sort_values("index")
error_labeled_meta.drop_duplicates(subset = "index", inplace=True)
error_labeled_meta.reset_index(drop=True, inplace=True)

train_data["index"] = summary["index"]
error_labeled_train = train_data.sort_values("index")
error_labeled_train.drop_duplicates(inplace=True)
error_labeled_train.drop(columns = ["index"], inplace=True)
error_labeled_train.reset_index(drop=True, inplace=True)

error_summary = summary.sort_values("index")

KeyError: 'index'

In [ ]:
model_eval = MachineLearningEvaluateModelStabilityNative(test_records=error_summary)
true_classes, stability_score = model_eval.run()
print("True classes: ", true_classes)
print("Model stability score: ", stability_score)

if (true_classes["class_true"] == "not_set").any():
    print("Classification Complete")
else:
    print("Some samples are unverified")

Model performance Summary:     index  threshold     score  confidence    class  \
0    158  -0.053316 -0.097339        1.83  outlier   
1    158  -0.066785 -0.078174        1.17  outlier   
2    158  -0.066785 -0.078174        1.17  outlier   
3    158  -0.066785 -0.078174        1.17  outlier   
4    158  -0.063543 -0.067164        1.06  outlier   

                         date class_true  stability_score  
0  2025-07-23 12-34-26-872389    outlier         0.902951  
1  2025-07-23 14-49-28-169722                    0.902951  
2  2025-07-23 14-49-20-912694                    0.902951  
3  2025-07-23 15-36-48-666559                    0.902951  
4  2025-07-24 09-39-18-957099                    0.902951  
True classes:        class_true majority_class  confidence_consistency
index                                                  
158      outlier        outlier                0.805902
159      not set        outlier                0.930314
160      outlier        outlier                0

In [ ]:
# sorted
true_classes.reset_index(drop=True, inplace=True)
error_labeled_meta["label"] = true_classes["class_true"]

print("labeled train: ", error_labeled_train)
print("labeled meta: ", error_labeled_meta)

labeled train:             area  pressure_max  pressure_min  pressure_mean  pressure_std  \
0   2025.952931       369.480           0.0     289.438981     68.443165   
1   1942.360731       342.210           0.0     277.494652     61.659245   
2   1926.345975       338.390           0.0     275.226188     60.570968   
3   1920.020041       337.115           0.0     274.322587     60.301285   
4   1916.016839       336.255           0.0     273.729984     60.039811   
5   1956.149018       347.295           0.0     279.462851     58.885165   
6   1987.490302       355.610           0.0     283.942890     65.331187   
7   1923.331503       337.895           0.0     274.776129     60.578398   
8   1907.835163       333.570           0.0     272.561176     59.728982   
9   2034.864198       355.580           0.0     290.708789     64.058379   
10  2072.595282       364.380           0.0     296.100207     66.814175   
11  2080.878308       366.945           0.0     297.305057     67.448580

In [ ]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodNearestNeighboursClassifierSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

ana = MachineLearningAnalyses(variables=error_labeled_train, metadata=error_labeled_meta)

scl = MachineLearningScaleFeaturesScalerSklearn()
ana = scl.run(ana)

knn = MachineLearningMethodNearestNeighboursClassifierSklearn()
ana = knn.run(ana)

print(ana)


NearestNeighboursAnalyses 
  variables: 20 rows, 16 columns
  metadata: 20 rows, 15 columns



In [ ]:
ana.plot_data()

In [ ]:
train_classes = ana.get_training_scores()
print(train_classes)

['outlier' 'outlier' 'outlier' 'outlier' 'outlier' 'outlier' 'outlier'
 'outlier' 'outlier' 'outlier' 'outlier' 'outlier' 'outlier' 'outlier'
 'outlier' 'outlier' 'outlier' 'outlier' 'outlier' 'outlier']


In [ ]:
classes = ana.get_prediction_classes()
print(classes)

None
